## Code from: 
    https://github.com/balzer82/ICINCO-2014/blob/master/.ipynb_checkpoints/Extended-Kalman-Filter-CTRV-Attitude-TestData2-checkpoint.ipynb

#### Search #param for all the parameters to change depends on the scenario

In [20]:
import numpy as np
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.stats import norm
import time
from IPython.display import Image as ImageDisp
from sympy import Symbol, symbols, Matrix, sin, cos, latex#, Plot
from sympy.interactive import printing
printing.init_printing()
%pylab inline --no-import-all

Populating the interactive namespace from numpy and matplotlib


In [9]:
adaptive = True

In [10]:
numstates = 6 

In [11]:
dt    = 1.0/10.0 # Sample Rate of the Measurements is 50Hz
dtGPS = 1.0/10.0 # Sample Rate of GPS is 10Hz

In [14]:
vs, psis, dpsis, dts, xs, ys, axs, phis, dphis, thetas, dthetas, Lats, Lons = \
 symbols('v \psi \dot\psi T x y a_x \phi \dot\phi \Theta \dot\Theta Lat Lon')

As = Matrix([[xs+(vs/dpsis)*(sin(psis+dpsis*dts)-sin(psis))],
             [ys+(vs/dpsis)*(-cos(psis+dpsis*dts)+cos(psis))],
             [vs + axs*dts],
             [psis+dpsis*dts],
             [phis+dphis*dts],
             [thetas+dthetas*dts]])
state = Matrix([xs,ys,vs,psis,phis,thetas])

## Dynamic Matrix

In [22]:
As

⎡    v⋅(-sin(\psi) + sin(T⋅\dot\psi + \psi))⎤
⎢x + ───────────────────────────────────────⎥
⎢                    \dot\psi               ⎥
⎢                                           ⎥
⎢    v⋅(cos(\psi) - cos(T⋅\dot\psi + \psi)) ⎥
⎢y + ────────────────────────────────────── ⎥
⎢                   \dot\psi                ⎥
⎢                                           ⎥
⎢                 T⋅aₓ + v                  ⎥
⎢                                           ⎥
⎢             T⋅\dot\psi + \psi             ⎥
⎢                                           ⎥
⎢             T⋅\dot\phi + \phi             ⎥
⎢                                           ⎥
⎣           T⋅\dot\Theta + \Theta           ⎦

In [23]:
state

⎡  x   ⎤
⎢      ⎥
⎢  y   ⎥
⎢      ⎥
⎢  v   ⎥
⎢      ⎥
⎢ \psi ⎥
⎢      ⎥
⎢ \phi ⎥
⎢      ⎥
⎣\Theta⎦

In [24]:
As.jacobian(state)

⎡      -sin(\psi) + sin(T⋅\dot\psi + \psi)  v⋅(-cos(\psi) + cos(T⋅\dot\psi + \
⎢1  0  ───────────────────────────────────  ──────────────────────────────────
⎢                    \dot\psi                               \dot\psi          
⎢                                                                             
⎢      cos(\psi) - cos(T⋅\dot\psi + \psi)   v⋅(-sin(\psi) + sin(T⋅\dot\psi + \
⎢0  1  ──────────────────────────────────   ──────────────────────────────────
⎢                   \dot\psi                                \dot\psi          
⎢                                                                             
⎢0  0                   1                                      0              
⎢                                                                             
⎢0  0                   0                                      1              
⎢                                                                             
⎢0  0                   0                           